Credit:
This work is heavily borrowed from the following source:
Author: BARAN BINGÖL
Kaggle URL: https://www.kaggle.com/code/baranbingl/face-shape-detection-85-acc-on-test-set

In [1]:
# !pip3 install numpy matplotlib pillow torch torchvision scikit-learn

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
os.environ["OMP_NUM_THREADS"] = "1"

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torch.cuda.amp import GradScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print(f"On device : {device}")
torch.cuda.empty_cache()

On device : cuda


In [4]:
datasets_path = '../datasets'

In [5]:
# oval_training_path = os.path.join(datasets_path, 'training_set/Oval')
# example_image_name = random.choice(os.listdir(oval_training_path))
# example_image_path = os.path.join(oval_training_path, example_image_name)
# example_image = Image.open(example_image_path)

# plt.imshow(example_image)
# plt.title(f"Image Size: {example_image.size}")
# plt.axis('off')
# plt.show()

In [6]:
train_transforms = T.Compose([
    T.Resize((224, 224)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(degrees=10),
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
BATCH_SIZE = 24
NUM_WORKERS = os.cpu_count() - 1
TRAIN_PATH = os.path.join(datasets_path, './training_set')
TEST_PATH = os.path.join(datasets_path, './testing_set')

def safe_pil_loader(path):
    try:
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')
    except (OSError, SyntaxError):
        return Image.new('RGB', (224, 224))

train_dataset = datasets.ImageFolder(root=TRAIN_PATH, transform=train_transforms, loader=safe_pil_loader)
test_dataset = datasets.ImageFolder(root=TEST_PATH, transform=test_transforms, loader=safe_pil_loader)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [8]:
# train_dataset.class_to_idx

In [9]:
# dataiter = iter(train_loader)
# images, labels = next(dataiter)

# example_image = images[0]
# example_image = example_image.permute(1, 2, 0).numpy()
# example_label = labels[0].item()

# mean = [0.485, 0.456, 0.406]
# std = [0.229, 0.224, 0.225]
# example_image = example_image * std + mean

# plt.imshow(example_image)
# plt.title(f"Label: {example_label}")
# plt.axis('off')
# plt.show()

In [10]:
torch.manual_seed(42)

model = torchvision.models.efficientnet_b4(pretrained=True)
num_classes = len(train_dataset.classes)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5, inplace=True),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, betas=(0.9, 0.999), weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

model.to(device)
num_epochs = 5
best_val_loss = 1_000_000

/home/lison/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lison/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/lison/.local/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [11]:
# scaler = GradScaler()

# for epoch in range(num_epochs):
#     model.train()
#     train_losses =  []

#     for inputs, labels in train_loader:

#         inputs, labels = inputs.to(device), labels.to(device)

#         optimizer.zero_grad()

#         with torch.autocast(device_type=device, dtype=torch.float16):
#           outputs = model(inputs)
#           loss = criterion(outputs, labels)

#         scaler.scale(loss).backward()

#         scaler.step(optimizer)
#         scaler.update()

#         train_losses.append(loss.item())

#     avg_train_loss = sum(train_losses) / len(train_losses)

#     model.eval()
#     validation_losses = []
#     all_predictions = []
#     all_labels = []

#     with torch.inference_mode():
#       for inputs, labels in test_loader:

#         inputs, labels = inputs.to(device), labels.to(device)

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         validation_losses.append(loss)

#         _, predicted = torch.max(outputs.data, 1)
#         all_predictions.extend(predicted.cpu().numpy())
#         all_labels.extend(labels.cpu().numpy())

#       avg_val_loss = sum(validation_losses) / len(validation_losses)
#       scheduler.step(avg_val_loss)

#       accuracy = accuracy_score(all_labels, all_predictions)
#       print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

#       if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         os.makedirs('models', exist_ok=True)
#         torch.save(model.state_dict(), 'models/best_model.pth')
# print(f'Best Validation Loss: {best_val_loss:.4f}')

In [12]:
def train_model(model, train_loader, criterion, optimizer, scheduler, device, num_epochs):
    train_losses = []
    train_accuracies = []

    if device.type == 'cuda':
        scaler = GradScaler()
        # print("Using Scaler")
    else:
        scaler = None
        # print("Not using Scaler")

    for epoch in range(num_epochs):
        model.train()
        epoch_train_losses = []
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            if scaler is not None:
                # print("Using Scaler")
                with torch.cuda.amp.autocast():
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                # print("Not using Scaler")
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            epoch_train_losses.append(loss.item())

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_train_loss = np.mean(epoch_train_losses)
        train_losses.append(avg_train_loss)

        accuracy = 100 * correct / total
        train_accuracies.append(accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {accuracy:.2f}%")

    # Plotting training loss and accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
    plt.plot(range(1, num_epochs + 1), train_accuracies, label='Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title('Training Loss and Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()


In [13]:
def test_model(model, test_loader, criterion, device):
    model.eval()
    test_losses = []
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_test_loss = np.mean(test_losses)
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions, average='macro')
    recall = recall_score(all_labels, all_predictions, average='macro')
    f1 = f1_score(all_labels, all_predictions, average='macro')

    print(f"Test Loss: {avg_test_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

In [14]:
def save_model(model, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)

In [15]:
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=device, abbreviated=False)
train_model(model, train_loader, criterion, optimizer, scheduler, device, num_epochs)

/home/lison/.local/lib/python3.10/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/5, Train Loss: 1.5775, Train Accuracy: 29.57%
Epoch 2/5, Train Loss: 1.4208, Train Accuracy: 43.65%
Epoch 3/5, Train Loss: 1.2528, Train Accuracy: 53.67%
Epoch 4/5, Train Loss: 1.1089, Train Accuracy: 62.85%


In [ ]:
test_model(model, test_loader, criterion, device)

In [ ]:
save_model(model, '../outputs/best_model_self.pth')

In [ ]:
# !pip3 uninstall -y numpy matplotlib pillow torch torchvision scikit-learn